In [1]:
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM 
import torch


d:\Florence-2\newenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
#Kiểm tra xem có GPU khả dụng hay không
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# Tải mô hình Florence-2-base và processor từ Hugging Face
model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-large", torch_dtype=torch_dtype, trust_remote_code=True).to(device)
processor = AutoProcessor.from_pretrained("microsoft/Florence-2-large", trust_remote_code=True)

d:\Florence-2\newenv\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PC\.cache\huggingface\hub\models--microsoft--Florence-2-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- configuration_

In [20]:

# Đọc ảnh từ file cục bộ
image_path = "download (1).jpg"  # Thay đường dẫn bằng đường dẫn ảnh của bạn
image = Image.open(image_path)

def run_example(task_prompt, text_input=None):
    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input
    inputs = processor(text=prompt, images=image, return_tensors="pt").to(device, torch_dtype)
    generated_ids = model.generate(
      input_ids=inputs["input_ids"],
      pixel_values=inputs["pixel_values"],
      max_new_tokens=1024,
      num_beams=3
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

    parsed_answer = processor.post_process_generation(generated_text, task=task_prompt, image_size=(image.width, image.height))

    print(parsed_answer)




In [21]:
prompt = "<MORE_DETAILED_CAPTION>"
run_example(prompt)

{'<MORE_DETAILED_CAPTION>': 'The image shows a young man with white hair playing an electric guitar on a stage. He is wearing a black shirt and appears to be in the middle of a performance. The stage is lit up with blue and purple lights, creating a dramatic atmosphere. The man is holding the guitar with both hands and is singing into a microphone. He looks focused and intense as he plays.'}
